In [1]:
import sympy
from sympy import Rational
import numpy as np
import pandas as pd
import math
from gurobipy import *

train_info = pd.read_csv(r'D:\学习\课题\inverse optimization data\train_info_multiOD_singleclass\train_info_single_class_multi_origin0619.csv')

#变量维度:车次，站间区间
di = len(train_info['TYPE'])
station_interval = list(range(151019789, 151019778, -1))

#灵敏度分析邻域半径
s = 0.0002

alpha_value_initial = 0.04#0.2#0.1282 
beta_value_initial = 0.03#0.2#0.1476 
gamma_value_initial = 0.02#0.2#0.1545 
g0_value_initial = 0.01#0.2#0.0042 

#上层决策变量值记录
upper_parameters_record = sympy.Matrix([alpha_value_initial, beta_value_initial, gamma_value_initial, g0_value_initial])
#上层目标值记录
upper_obj_record = []
#上层灵敏度目标值记录
upper_obj_sensitivity_record = []
#收敛准侧值记录
indicator_record = []

origins_input = [151019789, 151019787, 151019785, 151019784]
origins_interval_num = {151019789:11, 151019787:9, 151019785:7, 151019784:6}
userclass_input = {origins_input[0]:[0], origins_input[1]:[0], origins_input[2]:[0], origins_input[3]:[0]}

train_choice_set_input = {}
train_choice_set_input[origins_input[0]] = {0:list(range(8,47))}
train_choice_set_input[origins_input[1]] = {0:list(range(5,47))}
train_choice_set_input[origins_input[2]] = {0:list(range(5,47))}
train_choice_set_input[origins_input[3]] = {0:list(range(3,47))}

demand_input = {}
demand_input[origins_input[0]] = {0: 496}
demand_input[origins_input[1]] = {0: 530}
demand_input[origins_input[2]] = {0: 430}
demand_input[origins_input[3]] = {0: 551}

desired_arrival_time_input = {}
desired_arrival_time_input[origins_input[0]] = {0:30900}
desired_arrival_time_input[origins_input[1]] = {0:31500}
desired_arrival_time_input[origins_input[2]] = {0:30900}
desired_arrival_time_input[origins_input[3]] = {0:31500}

#出行时间
T = {}
for o in origins_input:
    for j in train_info['TYPE']:
        T[(o,j)] = train_info[train_info['TYPE'] == j]['TRIP_TIME_average_%d'%o].iloc[0]
        
#乘车时间
Tv = {151019789: 110, 151019788: 130, 151019787: 140, 151019786: 120, 151019785: 160, 151019784: 180, 151019783: 180, 151019782: 180, 151019781: 200, 151019780: 170, 151019779: 110}

#n0既有流量
n0 = {}
for j in train_info['TYPE']:
    for i in station_interval:
        n0[(j,i)] = train_info[train_info['TYPE'] == j]['%d'%i].values[0]

#TE和TL
TE = {}
TL = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            TE[(o,i,j)] = max(desired_arrival_time_input[o][i] - train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0], 0)
            TL[(o,i,j)] = max(train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0] - desired_arrival_time_input[o][i], 0)

In [2]:
#定义符号
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
g0 = sympy.Symbol('g0')

lamb = sympy.Symbol('lamb')
no_num = sympy.Symbol('no_num')

nis = {}
nis_keys = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis[(o,i,j)] = sympy.Symbol('n%d,%d,%d'%(o,i,j))
            nis_keys.append((o,i,j))
miu = {}
for o in origins_input:
    for i in userclass_input[o]:
        miu[(o,i)] = sympy.Symbol('u%d,%d'%(o,i))

y_observed_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            y_observed_li.append(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)][j-1])
y_observed = sympy.Matrix(y_observed_li)        
        
y_observed_dic = {}
for o in origins_input:
    y_observed_dic[o] = {}
    for i in userclass_input[o]:
        y_observed_dic[o][i] = sympy.Matrix(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)])

upper_parameters_value = {}
upper_parameters_value[alpha] = alpha_value_initial
upper_parameters_value[beta] = beta_value_initial
upper_parameters_value[gamma] = gamma_value_initial
upper_parameters_value[g0] = g0_value_initial

In [3]:
flow_nis_aggregate = sympy.zeros(di,len(station_interval))
flow_nis_aggregate_dic = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] = flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(nis[(o,i,j)], (1, origins_interval_num[o])))

for j in train_info['TYPE']:
    for i in station_interval:
        flow_nis_aggregate_dic[(j, i)] = flow_nis_aggregate[(j - 1),(151019789 - i)]  

#车厢内拥挤成本
def invehicle_crowding_cost(origin, train, Tv_dic, n0_dic, flow_dic, g0_para):
    crowding_cost = 0
    for i in range(origin, 151019778, -1):
        crowding_cost = crowding_cost + g0_para * Tv_dic[i] * (flow_dic[(train, i)] + n0_dic[(train, i)] - 256) / 275
    return crowding_cost

In [4]:
#lagrange函数对n的导数
Ln = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            Ln.append((sympy.log(nis[(o,i,j)]) + 1) + invehicle_crowding_cost(o, j, Tv, n0, flow_nis_aggregate_dic, g0) + alpha * T[(o,j)] + beta * TE[(o,i,j)] + gamma * TL[(o,i,j)] - miu[(o,i)])            
            
Noi = []
for o in origins_input:
    for i in userclass_input[o]:
        noi = 0
        for j in train_choice_set_input[o][i]:
            noi = noi + nis[(o,i,j)]
        Noi.append(noi) 

#分块矩阵A
A_li = []
for num in range(0,len(Ln)):
    ln = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                ln.append(sympy.diff(Ln[num],nis[(o,i,j)]))
    A_li.append(ln)
    
A = sympy.Matrix(A_li)

#分块矩阵B
B_li = []
for num in range(0,len(Ln)):
    lu = []
    for o in origins_input:
        for i in userclass_input[o]:
            lu.append(sympy.diff(Ln[num],miu[(o,i)]))
    B_li.append(lu)
    
B = sympy.Matrix(B_li)

#分块矩阵C
C_li = []
for num in range(0,len(Noi)):
    nn = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nn.append(sympy.diff(Noi[num],nis[(o,i,j)]))
    C_li.append(nn)  
    
C = sympy.Matrix(C_li)

#分块矩阵D
D_li = []
for num in range(0,len(Noi)):
    nu = []
    for o in origins_input:
        for i in userclass_input[o]:
            nu.append(sympy.diff(Noi[num],miu[(o,i)]))
    D_li.append(nu)        

D = sympy.Matrix(D_li) 

#分块矩阵N
N_li = []
for num in range(0,len(Ln)):
    lp = []
    lp.append(sympy.diff(Ln[num],alpha))
    lp.append(sympy.diff(Ln[num],beta))
    lp.append(sympy.diff(Ln[num],gamma))
    lp.append(sympy.diff(Ln[num],g0))
    N_li.append(lp)
    
N = sympy.Matrix(N_li)

epoch = 0
indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))

In [5]:
#cost_fuction
def train_cost(flow_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_dic = {}
    for o in origins_input:
        for i in userclass[o]:
            for j in train_choice_set[o][i]:           
                train_cost_dic[(o, i, j)] = invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]

    return train_cost_dic

#logit车次流量加载
def logit(origin, user, train_cost_dic, demand, choice_set):
    
    cost_exp_li = []
    for j in train_info['TYPE']:
        if j in choice_set:
            cost_exp_li.append(sympy.exp( - train_cost_dic[(origin, user, j)]))
        else:
            cost_exp_li.append(0)
            
    flow = (sympy.Matrix(cost_exp_li) / np.sum(cost_exp_li)) * demand 
    return flow

#多类用户logit加载
def multiorigin_multiclass_logit(userclass, train_cost_dic, demand_dic, train_choice_set):
    
    flow_multiorigin_multiclass_aggregate = sympy.zeros(di,len(station_interval))
    flow_multiorigin_multiclass_dic = {}
    flow_multiorigin_multiclass_aggregate_dic = {}
    
    for o in origins_input:
        flow_multiorigin_multiclass_dic[o] = {}
        for i in userclass[o]:
            flow_multiorigin_multiclass_dic[o][i] = logit(o, i, train_cost_dic, demand_dic[o][i], train_choice_set[o][i])
            flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] = flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multiorigin_multiclass_dic[o][i], (1, origins_interval_num[o])))
    
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_multiorigin_multiclass_aggregate_dic[(j, i)] = flow_multiorigin_multiclass_aggregate[(j - 1),(151019789 - i)]
    
    return flow_multiorigin_multiclass_aggregate, flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic

#cost_fuction_stochastic
def train_cost_stochastic(flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_stochastic_dic = {}
    for o in origins_input:
        train_cost_stochastic_dic[o] = {}
        for i in userclass[o]:
            flow_origin_class_dic = dict(zip(train_info['TYPE'], flow_multiorigin_multiclass_dic[o][i]))
            train_cost_stochastic_dic[o][i] = {}
            for j in train_info['TYPE']:  #保证矩阵运算时宽度一致
                if j in train_choice_set[o][i]:
                    train_cost_stochastic_dic[o][i][j] = (sympy.log(flow_origin_class_dic[j])+ 1) + invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_multiorigin_multiclass_aggregate_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]
                else:
                    train_cost_stochastic_dic[o][i][j] = no_num
                    
    return train_cost_stochastic_dic

#二分法求步长
def binary_search(equation):
    a = 0
    b = 1
    fa = equation.evalf(subs = {lamb:a})
    fb = equation.evalf(subs = {lamb:b})
    while a <= b:
        mid = (a + b) / 2
        fm = equation.evalf(subs = {lamb:mid})
        if abs(fm) < 0.1:
            #print("方程的根为",mid)
            break
        if fa * fm < 0:
            a = a
            b = mid
            fb = equation.evalf(subs = {lamb:b})
            #print("解在[",a,",",b,"]之间")
        elif fb * fm < 0:
            a = mid
            b = b
            fa = equation.evalf(subs = {lamb:a})
            #print("解在[",a,",",b,"]之间")
            
    return mid

In [6]:
def train_flow_assignment(userclass, train_choice_set, multi_demand, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    n = 0
    indicator_am = 1
    flow_dic_initial = {}
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_dic_initial[(j, i)] = 0
    cost_dic = train_cost(flow_dic_initial, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    flow_aggregate_x_new, flow_multi_dic_x_new, flow_aggregate_dic_x_new = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
    
    while (indicator_am > 0.0001) or (n <= 20):
    
        flow_multi_dic_x = flow_multi_dic_x_new
        flow_aggregate_dic_x = flow_aggregate_dic_x_new

        #车次成本更新
        cost_dic = train_cost(flow_aggregate_dic_x, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        #logit流量加载
        flow_aggregate_y, flow_multi_dic_y, flow_aggregate_dic_y = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
        
        #辅助流量计算（计算迭代步长）
        flow_aggregate_aux = sympy.zeros(di,len(station_interval))
        flow_multi_dic_aux = {}
        flow_aggregate_dic_aux = {}
        
        for o in origins_input:
            flow_multi_dic_aux[o] = {}
            for i in userclass[o]:
                flow_multi_dic_aux[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb
                flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] = flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_aux[o][i], (1, origins_interval_num[o])))

        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_aux[(j, i)] = flow_aggregate_aux[(j - 1),(151019789 - i)]
                
        #计算迭代步长
        train_cost_stochastic_aux_dic = train_cost_stochastic(flow_multi_dic_aux, flow_aggregate_dic_aux, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        equation_lamb = 0
        for o in origins_input:
            for i in userclass[o]:
                equation_lamb = equation_lamb + ((flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]).T * sympy.Matrix(list(train_cost_stochastic_aux_dic[o][i].values())))[0,0]
        #print('start')
        lamb_value_df = binary_search(equation_lamb)
        #print(lamb_value_df)
    
        #确定新的迭代起点
        flow_multi_dic_x_new = {}
        flow_aggregate_dic_x_new = {}
        flow_aggregate_x_new = sympy.zeros(di,len(station_interval))
        for o in origins_input:
            flow_multi_dic_x_new[o] = {}
            for i in userclass[o]:
                flow_multi_dic_x_new[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb_value_df
                flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] = flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_x_new[o][i], (1, origins_interval_num[o])))
    
        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_x_new[(j, i)] = flow_aggregate_x_new[(j - 1),(151019789 - i)]
        
        #收敛性检验
        numerator = 0
        denominator = 0
        for o in origins_input:
            for i in userclass[o]:
                numerator = numerator + ((flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]).T * (flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]))[0,0]
                denominator = denominator + (sympy.ones(1,di) * flow_multi_dic_x[o][i])[0,0]
        
        indicator_am = (numerator ** 0.5) / denominator
        n = n + 1
#         print(indicator_am)
#         print(n)
        if (indicator_am <= 0.00001) or (n >= 20):
            break
    print(indicator_am)
    print(n)
    
    cost_dic_x_new = train_cost(flow_aggregate_dic_x_new, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    
    return flow_multi_dic_x_new, flow_aggregate_x_new, flow_aggregate_dic_x_new, cost_dic_x_new

In [7]:
while (indicator >= 0.00015) or (epoch == 0):
    
    epoch = epoch + 1
    flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0])
    
    #到达率流量取值
    nis_value_dic = {}
    nis_value_li = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
                nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         
    nis_value_matrix = sympy.Matrix(nis_value_li)
    
    #上层目标值记录
    upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
    print(f'上层目标值均衡:{upper_obj_record[-1]}')
    
    #矩阵运算所有参数取值
    matrix_parameters_value_all = {}
    matrix_parameters_value_all.update(nis_value_dic)
    matrix_parameters_value_all.update(upper_parameters_value)
    
    #矩阵计算
    A_value = np.matrix(A.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    B_value = np.matrix(B.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    C_value = np.matrix(C.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    D_value = np.matrix(D.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    N_value = np.matrix(N.evalf(subs = matrix_parameters_value_all)).astype(np.float64)

    #梯度计算
    I11_value = A_value.I + A_value.I * B_value * (D_value - C_value * A_value.I * B_value).I * C_value * A_value.I
    Grad = - I11_value * N_value    
    
    #反应函数
    x = sympy.Matrix([alpha, beta, gamma, g0])
    x0 = sympy.Matrix([matrix_parameters_value_all[alpha], matrix_parameters_value_all[beta], matrix_parameters_value_all[gamma], matrix_parameters_value_all[g0]])
    y = nis_value_matrix + Grad * (x - x0)
    
    #转化为矩阵形式，lh为系数矩阵，rh为右端项
    lhs, rhs = sympy.linear_eq_to_matrix(y - y_observed, [alpha, beta, gamma, g0])
    #转化为numpy数组
    lh = np.array(lhs).astype(np.float64)
    rh = np.array(rhs).astype(np.float64)
    
    #上层规划模型
    upper_model = Model("Bilevel upper model")
    # 约束非凸
    upper_model.Params.NonConvex = 2
    #上层决策变量
    alpha_up = upper_model.addVar(lb = upper_parameters_value[alpha] - s, ub = upper_parameters_value[alpha] + s, vtype = GRB.CONTINUOUS, name = "alpha_up")
    beta_up = upper_model.addVar(lb = upper_parameters_value[beta] - s, ub = upper_parameters_value[beta] + s, vtype = GRB.CONTINUOUS, name = "beta_up")
    gamma_up = upper_model.addVar(lb = upper_parameters_value[gamma] - s, ub = upper_parameters_value[gamma] + s, vtype = GRB.CONTINUOUS, name = "gamma_up")
    g0_up = upper_model.addVar(lb = upper_parameters_value[g0] - s, ub = upper_parameters_value[g0] + s, vtype = GRB.CONTINUOUS, name = "g0_up")

    #上层目标函数
    upper_model.setObjective(quicksum((lh[i,0] * alpha_up + lh[i,1] * beta_up + lh[i,2] * gamma_up + lh[i,3] * g0_up - rh[i,0])**2 for i in range(lh.shape[0])) ,GRB.MINIMIZE)
    
    #约束
    upper_model.addConstr( alpha_up >= 0)
    upper_model.addConstr( beta_up >= 0)
    upper_model.addConstr( gamma_up >= 0)
    upper_model.addConstr( g0_up >= 0)
    
    upper_model.optimize()    
    
    #上层目标函数优化值
    upper_model_objvalue = upper_model.ObjVal
    #上层决策变量优化值
    upper_parameters_value = {}
    upper_parameters_value[alpha] = [v.x for v in upper_model.getVars() if v.varName == 'alpha_up'][0]
    upper_parameters_value[beta] = [v.x for v in upper_model.getVars() if v.varName == 'beta_up'][0]
    upper_parameters_value[gamma] = [v.x for v in upper_model.getVars() if v.varName == 'gamma_up'][0]
    upper_parameters_value[g0] = [v.x for v in upper_model.getVars() if v.varName == 'g0_up'][0]
    
    #计算收敛准侧的指标
    upper_parameters_record = upper_parameters_record.col_insert(upper_parameters_record.shape[1], sympy.Matrix([upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0]]))
    indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))
    indicator_record.append(indicator)
    upper_obj_sensitivity_record.append(upper_model_objvalue)
    print(f'上层目标值灵敏度:{upper_model_objvalue}')
    print(f'indicator:{indicator}')
    print(f'参数结果：{upper_parameters_value}')
    print(f'epoch:{epoch}')

0.000576253087270463
20
上层目标值均衡:226444.879932374
Academic license - for non-commercial use only - expires 2023-09-22
Using license file C:\Users\XP\gurobi.lic
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x26b747c9
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+05, 2e+07]
  QObjective range [1e+06, 3e+10]
  Bounds range     [1e-02, 4e-02]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free v

         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 3.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.46015126e+09 -2.07397522e+05  1.12e+03 6.77e+02  1.00e+06     0s
   1  -1.42553120e+06  1.02351763e+05  1.17e+00 7.09e-01  1.15e+03     0s
   2   3.35666807e+04  1.02690426e+05  5.32e-02 3.23e-02  9.49e+01     0s
   3   1.74516684e+05  1.94178359e+05  6.25e-03 3.79e-03  1.13e+03     0s
   4   1.90763971e+05  1.94670142e+05  1.27e-03 7.30e-04  2.34e+02     0s
   5   1.92029787e+05  1.94721876e+05  1.05e-03 5.05e-04  1.82e+0

   0   9.23016730e+08 -2.14061629e+05  1.07e+03 6.71e+02  1.00e+06     0s
   1   1.05537280e+06  8.70227373e+04  1.12e+00 7.05e-01  1.10e+03     0s
   2   1.33777344e+05  8.70883109e+04  5.37e-02 3.38e-02  8.17e+01     0s
   3   1.77978669e+05  1.81260808e+05  3.05e-03 1.92e-03  5.85e+02     0s
   4   1.80653486e+05  1.81435765e+05  9.48e-04 4.16e-04  1.25e+02     0s
   5   1.80928319e+05  1.81451111e+05  8.15e-04 2.79e-04  9.26e+01     0s
   6   1.81607635e+05  1.83053379e+05  3.04e-04 1.91e-04  1.97e+02     0s
   7   1.83009377e+05  1.83224144e+05  3.54e-05 1.20e-05  1.80e+00     0s
   8   1.83236925e+05  1.83236335e+05  1.91e-11 7.17e-08  7.38e-02     0s
   9   1.83236845e+05  1.83236808e+05  1.91e-17 1.26e-09  4.73e-03     0s
  10   1.83236842e+05  1.83236841e+05  0.00e+00 6.16e-10  1.44e-04     0s
  11   1.83236842e+05  1.83236842e+05  5.42e-20 1.35e-11  1.43e-07     0s
  12   1.83236842e+05  1.83236842e+05  5.42e-20 9.31e-10  1.43e-10     0s

Barrier solved model in 12 iterations

  12   1.70282603e+05  1.70282603e+05  0.00e+00 3.03e-10  5.36e-07     0s
  13   1.70282603e+05  1.70282603e+05  0.00e+00 9.87e-12  5.37e-10     0s

Barrier solved model in 13 iterations and 0.01 seconds
Optimal objective 1.70282603e+05

上层目标值灵敏度:170282.6027054066
indicator:0.000199999999999968
参数结果：{alpha: 0.03864895534125288, beta: 0.028000000000039774, gamma: 0.018000000000000804, g0: 0.011999999999999029}
epoch:10
0.00141474929393302
20
上层目标值均衡:169640.680700865
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x0faf4333
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 6e+06]
  QObjective range [5e+04, 2e+10]
  Bounds range     [1e-02, 4e-02]
  RHS range        [0e+00, 0e

Model fingerprint: 0x1024c705
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+03, 6e+06]
  QObjective range [2e+03, 2e+10]
  Bounds range     [1e-02, 4e-02]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 3.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.77821310e+08 -9.11835699e+04  1.04e+03 4.03e+02  1.00e+06     0s
   1   7.58535619e+05  8.55229348e+04  1.21e+00 4.70e-01  1.17e+03     0s
   2   1.81570764e+05  8

...

Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+06, 4e+07]
  QObjective range [2e+07, 2e+11]
  Bounds range     [2e-04, 3e-03]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 3.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.23909609e+10 -3.21929257e+05  1.00e+03 3.82e+02  1.01e+06     0s
   1   1.24010216e+07 -8.96968421e+04  1.00e+00 3.83e-01  1.26e+04     0s
   2   4.51709850e+04  2.81784754e+03  3.37e-03 1.29e-

   9   4.32322312e+03  4.32322312e+03  5.61e-18 2.41e-09  1.40e-09     0s

Barrier solved model in 9 iterations and 0.01 seconds
Optimal objective 4.32322312e+03

上层目标值灵敏度:4323.223118410031
indicator:0.000199999999996709
参数结果：{alpha: 0.0021098642669383954, beta: 0.0006376333488019661, gamma: 0.0005721019679108802, g0: 2.398056502083311e-05}
epoch:200
9.09633024514717e-6
5
上层目标值均衡:4322.63959209359
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xbc944f06
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+06, 4e+07]
  QObjective range [2e+07, 2e+11]
  Bounds range     [2e-04, 2e-03]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus par

   0   1.22702014e+10 -3.20605019e+05  1.00e+03 3.79e+02  1.01e+06     0s
   1   1.22779882e+07 -8.97302894e+04  1.00e+00 3.79e-01  1.26e+04     0s
   2   3.52678284e+04  2.85658469e+03  2.60e-03 9.85e-04  6.33e+01     0s
   3   9.24144719e+03  3.17377331e+03  4.85e-04 1.84e-04  1.51e+01     0s
   4   8.16070402e+03  3.21560288e+03  3.97e-04 1.50e-04  1.34e+01     0s
   5   7.61610129e+03  3.86735723e+03  3.12e-04 1.18e-04  9.85e+01     0s
   6   3.99565876e+03  3.95681011e+03  3.98e-06 1.51e-06  3.67e-01     0s
   7   3.95896592e+03  3.95892244e+03  4.04e-09 1.53e-09  8.94e-04     0s
   8   3.95892859e+03  3.95892859e+03  4.04e-15 4.28e-10  8.94e-07     0s
   9   3.95892859e+03  3.95892859e+03  3.66e-18 2.64e-09  8.94e-10     0s

Barrier solved model in 9 iterations and 0.01 seconds
Optimal objective 3.95892859e+03

上层目标值灵敏度:3958.928591908045
indicator:0.000199999999997550
参数结果：{alpha: 0.0013098642669494523, beta: 0.0006250804971311087, gamma: 0.0005376784621955769, g0: 2.450892544939

  Bounds range     [2e-04, 9e-04]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 4 columns, 0 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 3.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.22623947e+10 -3.20279681e+05  1.00e+03 3.78e+02  1.01e+06     0s
   1   1.22689406e+07 -8.97261550e+04  1.00e+00 3.78e-01  1.26e+04     0s
   2   2.97778554e+04  2.90156437e+03  2.16e-03 8.15e-04  5.31e+01     0s
   3   7.70349765e+03  3.18989423e+03  3.61e-04 1.36e-04  1.10e+01     0s
   4   6.87841713e+03  3.22110768e+03  2.94e-04 1.11e-04  9.83e+00     


Barrier solved model in 10 iterations and 0.01 seconds
Optimal objective 3.51482123e+03

上层目标值灵敏度:3514.821231909047
indicator:0.000109864267045425
参数结果：{alpha: 6.918939811565186e-17, beta: 0.0006035361113038573, gamma: 0.00047679266893096945, g0: 2.3612107666500075e-05}
epoch:211


In [8]:
np.savetxt('indicator_record_multiOD_singleclass_sue_0.04321_0620.txt', np.array(indicator_record))
np.savetxt('upper_obj_record_multiOD_singleclass_sue_0.04321_0620.txt', np.array(upper_obj_record))
np.savetxt('upper_parameters_record_multiOD_singleclass_sue_0.04321_0620.txt', np.array(upper_parameters_record))
np.savetxt('upper_obj_sensitivity_record_multiOD_singleclass_sue_0.04321_0620.txt', np.array(upper_obj_sensitivity_record))